In [79]:
# Conventionally people rename the pandas import to pd for brevity
import pandas as pd

In [109]:
# Load in the data and preview it
sales = pd.read_csv('home_data.csv') 
sales.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [146]:
# Order by zipcode
sales = sales.sort_values('zipcode')

# Create a list of the unique zipcodes (numpy.ndarray)
unique_zipcodes = sales['zipcode'].unique()

# Create list of dataframes by zipcode
list_of_df = []

for zipcode in unique_zipcodes:
   # Create df for each zipcode
   df = sales[sales['zipcode'] == zipcode]
   # Append to the list_of_df
   list_of_df.append(df)

(70,)

In [163]:
from sklearn.model_selection import train_test_split

list_of_df_train = []
list_of_df_test = []

# Split each dataframe into train (80%) and test data (20%) 
for df in list_of_df:
   train_data, test_data = train_test_split(df, test_size=0.2)
   list_of_df_train.append(train_data)
   list_of_df_test.append(test_data)

print(f"# of zipcodes:", len(unique_zipcodes))
print(f"# of (train) dataframes:", len(list_of_df_train))
print(f"# of (test) dataframes:", len(list_of_df_test))


# of zipcodes: 70
# of (train) dataframes: 70
# of (test) dataframes: 70


In [165]:
from sklearn import linear_model

# Basic Model
y = train_data.price # price column of training set
X = train_data[basic_features]

# Create and train the model
basic_model = linear_model.LinearRegression().fit(X, y)
basic_features = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors']
# Print the model
print(f"Basic Model \n Intercept: {basic_model.intercept_}")
df = pd.DataFrame(zip(basic_features, basic_model.coef_), columns=['Features','Coef'])
print(df)

print(X)

Basic Model 
 Intercept: -39191.12823594711
      Features          Coef
0     bedrooms -51638.162563
1    bathrooms  41521.963225
2  sqft_living    316.481308
3     sqft_lot     29.829980
4       floors  42065.076319
       bedrooms  bathrooms  sqft_living  sqft_lot  floors
18516         3       1.75         2520      4534     1.0
1360          3       2.00         2310      6000     2.0
8329          4       3.50         3080      4815     2.0
5843          4       1.75         1700      5846     1.0
20337         3       2.25         1230       875     3.0
...         ...        ...          ...       ...     ...
1050          3       2.00         1700      4250     1.0
16327         4       3.50         2730      5607     1.0
9021          3       2.25         1760      1800     2.0
4289          3       2.50         1910      1501     2.5
10969         4       2.50         2020      4350     2.0

[253 rows x 5 columns]


In [90]:
### edTest(test_train_rmse) ###

from sklearn.metrics import mean_squared_error 
# TODO

# Predicting prices
y = train_data.price # price column of training set
X = train_data[basic_features]
y_pred = basic_model.predict(X)
train_rmse_basic = mean_squared_error(y, y_pred, squared=False) #False = rmse

print(f"Basic Model RMSE (Training Data): {train_rmse_basic:.2f} \n")
print(y_pred)

print(X)
print(type(X))

Basic Model RMSE (Training Data): 251868.63 

[461436.71584674 356359.1551281  779128.59273735 ... 443765.59804266
 224481.01016227 422628.3292414 ]
       bedrooms  bathrooms  sqft_living  sqft_lot  floors
9464          4       2.50         1930      8660     1.0
11125         3       1.00         1440     11330     1.0
7047          3       2.50         2770      8820     1.0
10472         5       2.25         1950      8086     1.0
6869          4       2.50         1830      5175     2.0
...         ...        ...          ...       ...     ...
2329          4       1.75         2430     13211     1.5
8003          4       2.50         3070      6923     2.0
2187          3       2.50         1680     11312     1.0
3202          2       1.00          810      7980     1.0
17384         2       1.50         1430      1650     3.0

[17290 rows x 5 columns]
<class 'pandas.core.frame.DataFrame'>


In [91]:
# Predicting prices
y = test_data.price # price column of training set
X = test_data[basic_features]
y_pred = basic_model.predict(X)

test_rmse_basic = mean_squared_error(y, y_pred, squared=False) #False = rmse
print(f"Basic Model RMSE (Test Data): {test_rmse_basic:.2f} \n")


Basic Model RMSE (Test Data): 278435.60 



In [92]:
print(f"Basic RMSE Error: {train_rmse_basic:.2f} vs {test_rmse_basic:.2f}")


Basic RMSE Error: 251868.63 vs 278435.60


In [93]:
from joblib import dump, load
# Create the condensed model file
dump(basic_model, 'basic_model.joblib')

['basic_model.joblib']

In [100]:
import numpy as np
# Load the condensed model file
mdl = load('basic_model.joblib')

# Assume user input
userInput = {
   'bedrooms': 3,
   'bathrooms': 2,
   'sqft_living': 2200,
   'sqft_lot': 4000,
   'floors': 2
}

# Convert to pandas dataframe
df = pd.DataFrame([userInput])

# Predict based on input
y_pred = mdl.predict(df)

# Display the results
print(y_pred)


[602008.86649883]
